## Setup

In [1]:
df = read.csv('../data/cleaned.csv', check.names=FALSE)
df

AfterProbiotic,(12 or 13)-methylmyristate (a15:0 or i15:0),(14 or 15)-methylpalmitate (a17:0 or i17:0),(16 or 17)-methylstearate (a19:0 or i19:0),(3'-5')-adenylyladenosine*,(3'-5')-adenylylcytidine,(3'-5')-adenylyluridine,(3'-5')-cytidylylcytidine*,(3'-5')-cytidylylguanosine,(3'-5')-cytidylyluridine*,...,sedoheptulose,suberate (C8-DC),succinate,succinimide,threonate,tricarballylate,urate,xylose,mouseID,isMale
0,0.09769943,0.03126430,0.01175118,0.0025300513,0.0015953406,0.0025319685,0.0026998886,0.0015221590,0.0026410562,...,0.04072892,0.02473527,0.3473332,0.02198092,0.15567713,0.24143204,0.04836660,0.14906799,1,1
1,0.15894081,0.03345032,0.01087362,0.0006593277,0.0006593277,0.0006593277,0.0006593277,0.0006593277,0.0012362812,...,0.03418842,0.04274254,0.3746433,0.03385185,0.08829207,0.07861127,0.04443141,0.18018382,1,1
0,0.11959562,0.03428829,0.01158458,0.0014854743,0.0007846236,0.0016378315,0.0018177270,0.0007846236,0.0019728938,...,0.06084485,0.03310305,0.2866504,0.03065286,0.11693983,0.19700936,0.04518001,0.16832822,2,1
1,0.13091614,0.03704059,0.01544693,0.0009213574,0.0009213574,0.0009213574,0.0009213574,0.0009213574,0.0009213574,...,0.09177728,0.03599431,0.4429625,0.02220815,0.07813900,0.10542181,0.03251015,0.14004571,2,1
0,0.10378773,0.03746011,0.01454331,0.0025258186,0.0024364777,0.0027643357,0.0035661809,0.0007091002,0.0033885674,...,0.03437333,0.03238493,0.2864540,0.02873003,0.19599581,0.32846756,0.05767700,0.07274028,3,1
1,0.15449811,0.03694344,0.01405253,0.0033453494,0.0022560673,0.0035784999,0.0034529102,0.0023853646,0.0035252966,...,0.05892596,0.04487653,0.3616701,0.03120959,0.12471296,0.13902043,0.06309067,0.09322475,3,1
0,0.17824911,0.04016364,0.01589867,0.0030008185,0.0022350649,0.0031025425,0.0030461414,0.0009414246,0.0031171519,...,0.02181575,0.03059914,0.3447633,0.02945386,0.15395690,0.25346776,0.06358654,0.11890044,4,0
1,0.11162569,0.03075153,0.01109551,0.0008319971,0.0008319971,0.0008319971,0.0008319971,0.0008319971,0.0008319971,...,0.02158385,0.03351839,0.2456366,0.02058728,0.18538022,0.15985092,0.08296148,0.10155370,4,0
0,0.16020749,0.03896578,0.01305617,0.0028324050,0.0018931417,0.0032457082,0.0009337653,0.0009337653,0.0023826135,...,0.02633124,0.02768154,0.2404378,0.02345896,0.12133600,0.19704572,0.05242202,0.28736344,5,0
1,0.12925316,0.03119683,0.01179281,0.0010613359,0.0010613359,0.0010613359,0.0034758482,0.0010613359,0.0035674905,...,0.02390423,0.03044215,0.2163461,0.02434295,0.19916877,0.17463974,0.08163638,0.05505979,5,0


In [2]:
biochemicals = colnames(df)[2:824]
head(biochemicals)

[1] "(12 or 13)-methylmyristate (a15:0 or i15:0)"
[2] "(14 or 15)-methylpalmitate (a17:0 or i17:0)"
[3] "(16 or 17)-methylstearate (a19:0 or i19:0)" 
[4] "(3'-5')-adenylyladenosine*"                 
[5] "(3'-5')-adenylylcytidine"                   
[6] "(3'-5')-adenylyluridine"

## Setup helper functions for each model

#### Model 1: dependent variable is metabolite; independent variable is time-point.


In [3]:
get_p_val_m1 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX)]
    colnames(data)[2] = 'abundance'
    model = lm(abundance ~ AfterProbiotic, data=data, family=binomial)
    coef = coef(summary(model))[,4]
    p_val = coef[2]
    return (p_val)
}

In [4]:
# Test function on first biochemical
get_p_val_m1(df, 2)

Warning message:
“In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument ‘family’ will be disregarded”

AfterProbiotic 
     0.7842764

#### Model 2: dependent variable is metabolite; independent variables are time-point and mouse sex.

In [5]:
get_p_val_m2 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX, 826)]
    colnames(data)[2] = 'abundance'
    model = lm(abundance ~ AfterProbiotic + isMale, data=data, family=binomial)
    coef = coef(summary(model))[,4]
    p_val_time = coef[2]
    return (p_val_time)
}

In [6]:
# Test function on first biochemical
get_p_val_m2(df, 2)

Warning message:
“In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument ‘family’ will be disregarded”

AfterProbiotic 
     0.7872136

#### Model 3: dependent variable is metabolite; independent variables are time-point and mouse sex, and also control for mouseID as a random-effect (use 'lmer' function with (1|mouseID).

In [7]:
library(lmerTest)
get_p_val_m3 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX, 825, 826)]
    colnames(data)[2] = 'abundance'
    model = lmer(abundance ~ AfterProbiotic + isMale + (1|mouseID), data=data)
    coef = coef(summary(model))[,5]
    p_val_time = coef[2]
    return (p_val_time)
}

Warning message:
“package ‘lmerTest’ was built under R version 3.6.3”Loading required package: lme4
Loading required package: Matrix
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step



In [8]:
# Test function on first biochemical
get_p_val_m3(df, 2)

boundary (singular) fit: see ?isSingular


AfterProbiotic 
     0.7872136

In [9]:
get_p_vals = function(dataframe, model_func) {
    # dataframe is the dataframe with all the data.
    # model_func is the function that returns the p-values for the model of interest
    # returns the p-values associated with the time variable for each metabolite
    p_vals = data.frame(
        biochemical = biochemicals,
        p_val = -1
    )
    print(head(p_vals))
    for (biochemIDX in 2:824) {
        p_val = tryCatch(
          model_func(dataframe, biochemIDX),
          error=function(e) {return (1)}
        )
        p_vals[biochemIDX - 1, 2] = p_val
    }
    return (p_vals)
}

## Run the three models on the 823 metabolites

In [ ]:
p_vals_m1 = get_p_vals(df, get_p_val_m1)
p_vals_m2 = get_p_vals(df, get_p_val_m2)
p_vals_m3 = get_p_vals(df, get_p_val_m3)

In [11]:
head(p_vals_m3)

biochemical,p_val
(12 or 13)-methylmyristate (a15:0 or i15:0),0.7872136
(14 or 15)-methylpalmitate (a17:0 or i17:0),0.2954750
(16 or 17)-methylstearate (a19:0 or i19:0),0.6007737
(3'-5')-adenylyladenosine*,0.1158123
(3'-5')-adenylylcytidine,0.0801481
(3'-5')-adenylyluridine,0.1002768


## Analysis

In [12]:
p_vals_m3_adjusted = data.frame(p_vals_m3)
p_vals_m3_adjusted[, 2] = p.adjust(p_vals_m3_adjusted[,2], method='BH')
head(p_vals_m3_adjusted)

biochemical,p_val
(12 or 13)-methylmyristate (a15:0 or i15:0),0.9357310
(14 or 15)-methylpalmitate (a17:0 or i17:0),0.6064238
(16 or 17)-methylstearate (a19:0 or i19:0),0.8539495
(3'-5')-adenylyladenosine*,0.3841295
(3'-5')-adenylylcytidine,0.3381292
(3'-5')-adenylyluridine,0.3635794


In [13]:
p_vals_m3_sig = p_vals_m3_adjusted[p_vals_m3_adjusted[, 'p_val'] <= 0.20,]
ordered = p_vals_m3_sig[order(p_vals_m3_sig$p_val),]
print(nrow(ordered))
head(ordered)

[1] 118


,biochemical,p_val
344,5-methyl-2'-deoxycytidine,0.02028685
184,glycocholate sulfate,0.12446353
222,myristoleate (14:1n5),0.12446353
473,N-acetyl-3-methylhistidine*,0.12446353
696,palmitoleoyl ethanolamide*,0.12446353
710,pheophytin A,0.12446353


In [14]:
write.csv(ordered, '../output/p_vals_m3.csv', row.names=FALSE)

## Calculate pre/post means for significant metabolites

In [19]:
ordered_all = c(5-methyl-2'-deoxycytidine
myristoleate (14:1n5)
palmitoleoyl ethanolamide*
N-acetyl-3-methylhistidine*
pheophytin A
glycocholate sulfate
citrate
succinylcarnitine (C4-DC)
deoxycholate
serotonin
lithocholate
1-carboxyethylisoleucine
alpha-muricholate
pyridoxine (Vitamin B6)
malate
5-methyluridine (ribothymidine)
galactitol (dulcitol)
5-hydroxyhexanoate
xanthurenate
arachidonoyl ethanolamide
N-palmitoylglycine
2-isopropylmalate
N-formylmethionine
dihydoxyphenylalanine (L-DOPA)
glycosyl-N-stearoyl-sphingosine (d18:1/18:0)
ergothioneine
isobutyrylglycine
3-formylindole
methionine
2'-deoxycytidine
hydantoin-5-propionate
taurocholate
indoleacetate
2,8-quinolinediol
2-methylbutyrylglycine
kynurenate
mannitol/sorbitol
dihydrocaffeate sulfate (2)
N,N-dimethyl-5-aminovalerate
docosahexaenoate (DHA; 22:6n3)
1-palmitoyl-GPA (16:0)
docosapentaenoate (n3 DPA; 22:5n3)
valylglycine
S-(3-hydroxypropyl)mercapturic acid (HPMA)
enterodiol
tartronate (hydroxymalonate)
N-acetyl-isoputreanine*
carboxyethyl-GABA
7,12-diketolithocholate
ribitol
oleoyl ethanolamide
N-acetyl-beta-glucosaminylamine
cysteine
vanillactate
N6-succinyladenosine
3-(3-hydroxyphenyl)propionate
sarcosine
palmitoyl-oleoyl-glycerol (16:0/18:1) [2]*
caprylate (8:0)
vanillate
N,N,N-trimethyl-5-aminovalerate
pinosylvin
octadecanedioate (C18-DC)
diaminopimelate
L-urobilin
dihomo-linolenate (20:3n3 or n6)
N-carbamoylaspartate
1-palmitoyl-2-linoleoyl-digalactosylglycerol (16:0/18:2)*
3-hydroxysuberate
m-tyramine
tauro-beta-muricholate
isovalerylglycine
beta-muricholate
1-carboxyethylleucine
beta-sitosterol
beta-cryptoxanthin
carotene diol (3)
hyodeoxycholate
N-acetyltryptophan
dihydroferulate
palmitoyl-linoleoyl-glycerol (16:0/18:2) [2]*
stearoyl ethanolamide
cholesterol
N-acetylthreonine
3-hydroxyoctanoate
margaroyl ethanolamide*
indolelactate
N-stearoyl-sphingosine (d18:1/18:0)*
3b-hydroxy-5-cholenoic acid
valine
1-carboxyethylvaline
fucose
tricarballylate
nicotinamide riboside
pinosylvin monomethylether
palmitoyl ethanolamide
arabonate/xylonate
lactosyl-N-palmitoyl-sphingosine (d18:1/16:0)
2-butenoylglycine
1-palmitoyl-2-linoleoyl-galactosylglycerol (16:0/18:2)*
oleoyl-oleoyl-glycerol (18:1/18:1) [2]*
pyridoxal
o-Tyrosine
4-hydroxybenzoate
isoleucylglycine
alpha-ketoglutarate
glycylvaline
triethanolamine
N-glycolylneuraminate
homostachydrine*
linoleoyl-linolenoyl-glycerol (18:2/18:3) [1]*
diethanolamine
suberate (C8-DC)
glycerophosphoethanolamine
linoleoyl-linoleoyl-glycerol (18:2/18:2) [1]*
campesterol
3-(3-hydroxyphenyl)propionate sulfate
valerate (5:0)
(N(1) + N(8))-acetylspermidine
margaroylcarnitine (C17)*
3-(4-hydroxyphenyl)propionate
uridine
O-sulfo-L-tyrosine
N-linoleoylglycine
heneicosapentaenoate (21:5n3)
caproate (6:0)
16-hydroxypalmitate
mannonate*
pyridoxamine
desmosterol
gamma-glutamylhistidine
benzoate
hexanoylglycine
ceramide (d18:1/14:0, d16:1/16:0)*
linoleoyl-linolenoyl-glycerol (18:2/18:3) [2]*
choline phosphate
dodecenedioate (C12:1-DC)*
citrulline
argininate*
cysteine s-sulfate
gamma-tocotrienol
octadecadienedioate (C18:2-DC)*
isocaproylglycine
3-acetylphenol sulfate
5-methylcytosine
4-hydroxycinnamate
N-acetylleucine
cysteine sulfinic acid
1-methylhistamine
1-carboxyethylphenylalanine
N2-acetyllysine
N1,N12-diacetylspermine
N-acetyltaurine
6-beta-hydroxylithocholate
leucylglycine
bilirubin (Z,Z)
oleoyl-linoleoyl-glycerol (18:1/18:2) [2]
linolenoyl-linolenoyl-glycerol (18:3/18:3) [2]*
apigenin
2-hydroxyhippurate (salicylurate)
hydroxymethylpyrimidine
arachidonate (20:4n6)
N-formylanthranilic acid
5-hydroxyindoleacetate
palmitate (16:0)
fructose
threonylphenylalanine
N-acetyl-cadaverine
dihomo-linoleate (20:2n6)
2-methylcitrate/homocitrate
4-hydroxyhippurate
alpha-tocopherol acetate
histidine
4-methylhexanoylglycine
10-hydroxystearate
2-hydroxyquinoline
dihydroorotate
isoleucine
trimethylamine N-oxide
p-hydroxybenzaldehyde
2'-O-methylcytidine
N-acetylphenylalanine
glycerophosphoglycerol
N-acetylisoleucine
1-stearoyl-2-docosahexaenoyl-GPC (18:0/22:6)
3-ureidoisobutyrate
phenylacetate
isohyodeoxycholate
2'-deoxyinosine
indole-3-carboxylate
o-cresol sulfate
phenol glucuronide
2-hydroxyoctanoate
p-cresol glucuronide*
(3'-5')-adenylylcytidine
gamma-glutamyl-epsilon-lysine
arginine
2-aminoadipate
glycylisoleucine
7-alpha-hydroxy-3-oxo-4-cholestenoate (7-Hoca)
biliverdin
1-(3-aminopropyl)-2-pyrrolidone
hypotaurine
nicotinate
pyridoxate
nicotinamide N-oxide
1-palmitoleoylglycerol (16:1)*
aspartate
N-acetylasparagine
5,6-dihydrothymine
N-acetylhistamine
2'-deoxyguanosine 5'-monophosphate (dGMP)
N6,N6-dimethyllysine
gamma-glutamylphenylalanine
eicosapentaenoate (EPA; 20:5n3)
N2,N6-diacetyllysine
D-urobilin
methylmalonate (MMA)
N-methylproline
N-methylalanine
cholate sulfate
linoleoyl ethanolamide
carotene diol (2)
dimethylguanidino valeric acid (DMGV)*
oleoyl-linolenoyl-glycerol (18:1/18:3) [2]*
(3'-5')-adenylyluridine
ribulonate/xylulonate*
thymidine 5'-monophosphate
1-(1-enyl-stearoyl)-GPE (P-18:0)*
9,10-DiHOME
isovalerate (i5:0)
1-palmitoyl-2-arachidonoyl-GPC (16:0/20:4n6)
1-methyl-beta-carboline-3-carboxylic acid
quinate
3-hydroxyadipate*
cytidine
2-myristoylglycerol (14:0)
glutaminylleucine
3-methylcrotonylglycine
1-methyladenine
(3'-5')-uridylyluridine
1-linoleoyl-2-arachidonoyl-GPC (18:2/20:4n6)*
3-methyladipate
C-glycosyltryptophan
N-acetyl-2-aminooctanoate*
1-oleoyl-GPA (18:1)
3-hydroxyhexanoate
(3'-5')-adenylyladenosine*
choline
phenylalanine
dihydrobiopterin
N6-methyladenosine
retinol (Vitamin A)
S-1-pyrroline-5-carboxylate
tryptophan
N-monomethylarginine
3-hydroxydecanoate
N-acetylvaline
5-hydroxylysine
ferulate
stigmasterol
sphingadienine
arachidoylcarnitine (C20)*
coprostanol
gamma-glutamyl-alpha-lysine
N-acetylmethionine
dimethylarginine (SDMA + ADMA)
N6-formyllysine
3-hydroxyoleate*
syringic acid
2-palmitoleoylglycerol (16:1)*
tigloylglycine
1-carboxyethyltyrosine
valerylglycine
1-(1-enyl-palmitoyl)-2-arachidonoyl-GPE (P-16:0/20:4)*
glycosyl-N-palmitoyl-sphingosine (d18:1/16:0)
1-docosahexaenoylglycerol (22:6)
thiamin (Vitamin B1)
salicylate
trans-nonadecenoate (tr 19:1)*
(3'-5')-guanylylcytidine
azelate (C9-DC)
1-linoleoylglycerol (18:2)
lactate
cystathionine
1,2-dilinoleoyl-digalactosylglycerol (18:2/18:2)*
linolenoyl ethanolamide
imidazole propionate
N6-acetyllysine
2-aminophenol sulfate
2-hydroxy-3-methylvalerate
1-dihomo-linolenylglycerol (20:3)
hyocholate
N-acetylglutamine
1-linolenoylglycerol (18:3)
N-acetylglutamate
glutamate, gamma-methyl ester
pyrraline
2-aminoheptanoate
eicosenoate (20:1)
ethylmalonate
1-myristoylglycerol (14:0)
2-stearoyl-GPE (18:0)*
erucate (22:1n9)
methylphosphate
1-stearoyl-2-arachidonoyl-GPC (18:0/20:4)
sinapate
1-stearoyl-GPC (18:0)
12,13-DiHOME
bilirubin (E,E)*
3-hydroxyisobutyrate
7-methylguanine
dihydroferulic acid sulfate
1-(1-enyl-palmitoyl)-GPE (P-16:0)*
N-acetylsphingosine
1-methyl-4-imidazoleacetate
1-oleoyl-GPG (18:1)*
imidazole lactate
5-dodecenoylcarnitine (C12:1)
ergosterol
palmitoyl-arachidonoyl-glycerol (16:0/20:4) [2]*
10-nonadecenoate (19:1n9)
protoporphyrin IX
phenyllactate (PLA)
carotene diol (1)
3-hydroxy-2-ethylpropionate
ferulylglycine (2)
N-acetylhistidine
glutarate (C5-DC)
pheophorbide A
mannose
phenylalanylglycine
ursocholate
serine
sulfate*
glucose
2-aminooctanoate
pantothenate
cadaverine
histidine betaine (hercynine)*
2-linoleoylglycerol (18:2)
1-(1-enyl-palmitoyl)-2-linoleoyl-GPE (P-16:0/18:2)*
1-oleoyl-GPE (18:1)
2'-deoxyadenosine
3-hydroxybutyrate (BHBA)
proline
4-acetamidobutanoate
1-palmitoyl-2-docosahexaenoyl-GPC (16:0/22:6)
pantetheine
5-oxoproline
glycylleucine
adrenate (22:4n6)
glutamate
orotate
eicosanoylsphingosine (d20:1)*
butyrylglycine
guanosine 5'- monophosphate (5'-GMP)
glucosylglycerol
maltotriose
4-acetylphenol sulfate
alpha-hydroxyisocaproate
glycerate
1,2-dilinoleoyl-galactosylglycerol (18:2/18:2)*
2-palmitoylglycerol (16:0)
10-heptadecenoate (17:1n7)
cysteinylglycine
5-methylthioribose**
allantoin
3-hydroxy-3-methylglutarate
isocitric lactone
N-acetylalanine
2-hydroxynervonate*
N-stearoyl-sphinganine (d18:0/18:0)*
tyrosol
N-linoleoyltaurine*
N-acetyl-1-methylhistidine*
1-palmitoylglycerol (16:0)
docosatrienoate (22:3n6)*
3-dehydroshikimate
hydroxyasparagine**
maltose
1-dihomo-linoleoylglycerol (20:2)
gluconate
leucylalanine
thymidine
1-oleoylglycerol (18:1)
3-sulfo-L-alanine
glycerophosphorylcholine (GPC)
alpha-hydroxyisovalerate
glyco-beta-muricholate**
N-acetylpyrraline
1-stearoyl-GPG (18:0)
myristate (14:0)
tryptophylglycine
quinolinate
1-(1-enyl-stearoyl)-2-arachidonoyl-GPE (P-18:0/20:4)*
sedoheptulose
soyasaponin III
1-linoleoyl-GPE (18:2)*
5-(2-Hydroxyethyl)-4-methylthiazole
(14 or 15)-methylpalmitate (a17:0 or i17:0)
1-margaroylglycerol (17:0)
glycosyl ceramide (d16:1/24:1, d18:1/22:1)*
guanine
2-oleoylglycerol (18:1)
arabinose
valylleucine
S-methylcysteine
gamma-glutamylleucine
1-methylguanidine
histidylalanine
valylglutamine
betaine
alanylleucine
2-hydroxystearate
4-thiouracil
2'-deoxyuridine
(3'-5')-cytidylyluridine*
octadecanedioylcarnitine (C18-DC)*
7-ketolithocholate
cytidine 2' or 3'-monophosphate (2' or 3'-CMP)
2,4,6-trihydroxybenzoate
dihydrokaempferol
cytosine
alpha-tocotrienol
sphingosine
xylose
sphinganine
N-delta-acetylornithine
glycodeoxycholate
naringenin
linoleoyl-arachidonoyl-glycerol (18:2/20:4) [2]*
1-(1-enyl-palmitoyl)-2-arachidonoyl-GPC (P-16:0/20:4)*
2-hydroxyadipate
13-HODE + 9-HODE
1-stearoyl-2-linoleoyl-GPE (18:0/18:2)*
phytosphingosine
equol
gamma-glutamylisoleucine*
linoleoyl-docosahexaenoyl-glycerol (18:2/22:6) [2]*
oleoyl-arachidonoyl-glycerol (18:1/20:4) [2]*
1-(1-enyl-palmitoyl)-2-oleoyl-GPE (P-16:0/18:1)*
arachidoyl ethanolamide (20:0)*
guanosine
octadecenedioate (C18:1-DC)*
4-hydroxymandelate
palmitoleoyl-linoleoyl-glycerol (16:1/18:2) [1]*
palmitoyl-linolenoyl-glycerol (16:0/18:3) [2]*
glycerol
N-oleoyl-sphingosine (d18:1/18:1)*
5,6-dihydrouridine
6-oxolithocholate
1-stearoyl-GPE (18:0)
oleoylcholine
urate
sphingomyelin (d18:1/14:0, d16:1/16:0)*
gamma-tocopherol/beta-tocopherol
pipecolate
allantoic acid
glucuronate
1,2-dipalmitoyl-GPG (16:0/16:0)
gentisate
deoxymugineic acid
1-linoleoyl-2-linolenoyl-GPC (18:2/18:3)*
gamma-glutamylglycine
ornithine
alpha-ketoglutaramate*
N-acetylglycine
phenylpyruvate
4-methylcatechol sulfate
phenol sulfate
malonate
prolylglycine
1,2-dilinolenoyl-digalactosylglycerol (18:3/18:3)
isocaproylglutamine
nisinate (24:6n3)
3-ureidopropionate
adenosine
myristoylcarnitine (C14)
hexadecanedioate (C16-DC)
gamma-glutamylalanine
glycine
linoleoyl-docosahexaenoyl-glycerol (18:2/22:6) [1]*
2-hydroxypalmitate
anthranilate
equol sulfate
phenylacetylglutamine
pimelate (C7-DC)
1-palmitoyl-2-stearoyl-GPC (16:0/18:0)
allo-threonine
N-acetyltyrosine
1-palmitoyl-2-linoleoyl-GPI (16:0/18:2)
argininosuccinate
nicotinate ribonucleoside
pseudouridine
lysine
phenylalanylalanine
3-(4-hydroxyphenyl)lactate
gamma-glutamylmethionine
lysylleucine
dipicolinate
riboflavin (Vitamin B2)
asparagine
N-(2-furoyl)glycine
7-ketodeoxycholate
indoleacetylglycine
1-methylnicotinamide
delta-tocopherol
myo-inositol
glycocholate
ribulose/xylulose
indolin-2-one
dodecanedioate (C12-DC)
heptadecasphingosine (d17:1)
diacetylchitobiose
N1-methyladenosine
laurylcarnitine (C12)
N-acetylmuramate
(3'-5')-uridylylcytidine*
4-cholesten-3-one
orotidine
succinate
agmatine
alpha-ketobutyrate
2-hydroxylignocerate*
sphingomyelin (d18:2/24:1, d18:1/24:2)*
docosadienoate (22:2n6)
leucylglutamine*
feruloylputrescine
gamma-glutamylvaline
beta-alanine
1-stearoyl-2-linoleoyl-GPC (18:0/18:2)*
N-behenoyl-sphingadienine (d18:2/22:0)*
linoleoylcholine*
soyasaponin II
2-oxoarginine*
alpha-tocopherol
3-methylglutarate/2-methylglutarate
1-palmitoyl-GPE (16:0)
2-hydroxyarachidate*
5-aminovalerate
glutarylcarnitine (C5-DC)
3-dehydrocholate
N-acetylaspartate (NAA)
ferulic acid 4-sulfate
sphingomyelin (d18:1/21:0, d17:1/22:0, d16:1/23:0)*
soyasaponin I
(3'-5')-cytidylylcytidine*
6-hydroxyindole sulfate
fumarate
N-acetylarginine
N-acetylglucosamine/N-acetylgalactosamine
1-stearoyl-2-oleoyl-GPE (18:0/18:1)
indolepropionylglycine
hippurate
1-lignoceroyl-GPC (24:0)
erythronate*
glycerol 3-phosphate
hexadecasphinganine (d16:0)*
N-alpha-acetylornithine
N1-Methyl-4-pyridone-3-carboxamide
adipate (C6-DC)
threonine
N1-Methyl-2-pyridone-5-carboxamide
flavin adenine dinucleotide (FAD)
diacylglycerol (16:1/18:2 [2], 16:0/18:3 [1])*
p-cresol sulfate
eicosenoylcarnitine (C20:1)*
tricosanoyl sphingomyelin (d18:1/23:0)*
1,5-anhydroglucitol (1,5-AG)
heptenedioate (C7:1-DC)*
octadecenedioylcarnitine (C18:1-DC)*
maleate
3-methyl-2-oxovalerate
1-(1-enyl-palmitoyl)-2-palmitoyl-GPC (P-16:0/16:0)*
3-hydroxypyridine sulfate
1,2-dipalmitoyl-GPE (16:0/16:0)*
N-acetylcysteine
(16 or 17)-methylstearate (a19:0 or i19:0)
aconitate [cis or trans]
(3'-5')-cytidylylguanosine
guanidinoacetate
1-palmitoyl-2-docosahexaenoyl-GPE (16:0/22:6)*
dimethylmalonic acid
caffeic acid sulfate
1-pentadecanoylglycerol (15:0)
18-methylnonadecanoate (i20:0)
1-methylguanine
2-hydroxybehenate
N-acetylproline
4-hydroxycinnamate sulfate
stearate (18:0)
1-(1-enyl-oleoyl)-GPE (P-18:1)*
behenate (22:0)*
threonate
2'-deoxyadenosine 5'-monophosphate
cholate
1-palmitoyl-2-linoleoyl-GPE (16:0/18:2)
adenine
1,2-dilinoleoyl-GPC (18:2/18:2)
leucine
guanosine-2',3'-cyclic monophosphate
2,3-dihydroxyisovalerate
ceramide (d16:1/24:1, d18:1/22:1)*
N6-carbamoylthreonyladenosine
N-palmitoyl-phytosphingosine (t18:0/16:0)
glycerophosphoinositol*
palmitoylcholine
stearoylcarnitine (C18)
deoxycarnitine
histamine
nicotianamine
pro-hydroxy-pro
phenethylamine
genistein sulfate*
1-(1-enyl-palmitoyl)-GPC (P-16:0)*
glycerophosphoserine*
mevalonate
1-linoleoyl-2-linolenoyl-digalactosylglycerol (18:2/18:3)*
N-palmitoyl-sphinganine (d18:0/16:0)
hypoxanthine
thymine
1-palmitoyl-2-linoleoyl-GPC (16:0/18:2)
xanthosine
nonadecanoate (19:0)
spermidine
alanine
erythrose
inosine
pentadecanoate (15:0)
hexadecadienoate (16:2n6)
N-acetylputrescine
4-methylthio-2-oxobutanoate
lignoceroyl sphingomyelin (d18:1/24:0)
tyrosine
1,2,3-benzenetriol sulfate (2)
4-methyl-2-oxopentanoate
gamma-glutamylglutamine
indolepropionate
enterolactone
3-hydroxysebacate
1-palmitoyl-2-oleoyl-GPC (16:0/18:1)
phenylpropionylglycine
flavin mononucleotide (FMN)
stearoyl sphingomyelin (d18:1/18:0)
1-(1-enyl-palmitoyl)-2-oleoyl-GPC (P-16:0/18:1)*
1-oleoyl-GPC (18:1)
beta-hydroxyisovalerate
tetradecadienoate (14:2)*
2'-O-methyluridine
gamma-glutamyltryptophan
3-methylhistidine
N-acetylmethionine sulfoxide
4-hydroxyphenylpyruvate
lanosterol
ceramide (d18:2/24:1, d18:1/24:2)*
ectoine
tyramine
1-stearoyl-GPS (18:0)*
ferulylglycine (1)
guaiacol sulfate
N-acetylserine
3-methyl-2-oxobutyrate
6-oxopiperidine-2-carboxylate
homocitrulline
cis-urocanate
phosphate
oleate/vaccenate (18:1)
ribose
pyroglutamine*
adipoylcarnitine (C6-DC)
biopterin
pyruvate
2,8-quinolinediol sulfate
oleoyl-linoleoyl-glycerol (18:1/18:2) [1]
daidzein sulfate (2)
daidzein
3-indoxyl sulfate
sphingomyelin (d18:1/20:0, d16:1/22:0)*
genistein
nicotinamide
erythritol
hydroxypalmitoyl sphingomyelin (d18:1/16:0(OH))**
1-stearoyl-2-oleoyl-GPC (18:0/18:1)
4-methoxyphenol sulfate
creatine
trans-urocanate
anserine
behenoyl sphingomyelin (d18:1/22:0)*
(12 or 13)-methylmyristate (a15:0 or i15:0)
1-palmitoyl-2-alpha-linolenoyl-GPC (16:0/18:3n3)*
N-palmitoyl-sphingosine (d18:1/16:0)
N-acetylglucosaminylasparagine
gulonate*
daidzein sulfate (1)
dimethylglycine
2-(4-hydroxyphenyl)propionate
cytidine 5'-monophosphate (5'-CMP)
sebacate (C10-DC)
hexadecenedioate (C16:1-DC)*
methionine sulfoxide
2-hydroxybutyrate/2-hydroxyisobutyrate
ribonate
3-hydroxystearate
thioproline
methylsuccinate
tryptamine
2-hydroxyglutarate
2'-deoxycytidine 5'-monophosphate
1-linoleoyl-2-linolenoyl-galactosylglycerol (18:2/18:3)*
palmitoylcarnitine (C16)
uridine-2',3'-cyclic monophosphate
1,2-dilinoleoyl-GPE (18:2/18:2)*
2-keto-3-deoxy-gluconate
palmitoleoylcarnitine (C16:1)*
arabitol/xylitol
sphingomyelin (d18:0/18:0, d19:0/17:0)*
3-phenylpropionate (hydrocinnamate)
cinnamoylglycine
dodecadienoate (12:2)*
undecanedioate (C11-DC)
3-hydroxybehenate*
cis-4-decenoate (10:1n6)*
palmitoyl-linoleoyl-glycerol (16:0/18:2) [1]*
ceramide (d18:1/20:0, d16:1/22:0, d20:1/18:0)*
stearidonate (18:4n3)
1-palmitoyl-GPG (16:0)*
N-methyl-GABA
mevalonolactone
linolenate [alpha or gamma; (18:3n3 or 6)]
arachidate (20:0)
N6-methyllysine
1-myristoyl-2-palmitoyl-GPC (14:0/16:0)
picolinate
kynurenine
linoleoylcarnitine (C18:2)*
trigonelline (N'-methylnicotinate)
4-guanidinobutanoate
4-vinylphenol sulfate
adenosine 5'-monophosphate (AMP)
N-acetylneuraminate
3-ketosphinganine
dehydrolithocholate
palmitoyl sphingomyelin (d18:1/16:0)
urea
palmitoleate (16:1n7)
N-acetyl-beta-alanine
diacetylspermidine*
gamma-glutamylthreonine
formiminoglutamate
xanthine
hexadecatrienoate (16:3n3)
linoleate (18:2n6)
pinitol
N-(2-hydroxypalmitoyl)-sphingosine (d18:1/16:0(2OH))
hexadecasphingosine (d16:1)*
1-methylhistidine
succinimide
malonylcarnitine
N-acetylcitrulline
gamma-glutamylglutamate
carnitine
isopimaric acid
creatinine
1-palmitoyl-2-palmitoleoyl-GPC (16:0/16:1)*
sphingomyelin (d18:1/17:0, d17:1/18:0, d19:1/16:0)
N6,N6,N6-trimethyllysine
docosapentaenoate (n6 DPA; 22:5n6)
3-amino-2-piperidone
1-oleoyl-2-linoleoyl-GPC (18:1/18:2)*
1-palmitoyl-2-arachidonoyl-GPE (16:0/20:4)*
pterin
trans-4-hydroxyproline
3-methylglutaconate
dihomo-linolenoylcarnitine (C20:3n3 or 6)*
sphingomyelin (d18:1/22:1, d18:2/22:0, d16:1/24:1)*
putrescine
(3'-5')-guanylyluridine
lignoceroyl ethanolamide (24:0)*
margarate (17:0)
3-hydroxyphenylacetate
palmitoyl dihydrosphingomyelin (d18:0/16:0)*
gamma-glutamyltyrosine
4-hydroxyphenylacetate
1-(1-enyl-palmitoyl)-2-linoleoyl-GPC (P-16:0/18:2)*
oleoylcarnitine (C18:1)
behenoyl ethanolamide (22:0)*
4-acetylcatechol sulfate (1)
1-stearoyl-GPI (18:0)
1-palmitoyl-GPC (16:0)
beta-guanidinopropanoate
biotin
catechol sulfate
1-linoleoyl-GPC (18:2)
1-ribosyl-imidazoleacetate*
stearoyl-linoleoyl-glycerol (18:0/18:2) [2]*
4-imidazoleacetate
1-heptadecenoylglycerol (17:1)*
sphingomyelin (d18:1/24:1, d18:2/24:0)*
cytidine 2',3'-cyclic monophosphate
3-aminoisobutyrate
taurine
heptadecatrienoate (17:3)*
glycitein
1,2-dipalmitoyl-GPC (16:0/16:0)
propionylglycine
2-oxindole-3-acetate
behenoyl dihydrosphingomyelin (d18:0/22:0)*
linoleoyl-arachidonoyl-glycerol (18:2/20:4) [1]*
myristoleoylcarnitine (C14:1)*
galactonate
sphingomyelin (d17:1/16:0, d18:1/15:0, d16:1/17:0)*
stachydrine
4-hydroxyphenylacetate sulfate¬†¬†¬†¬†¬†¬†¬†¬†
2'-deoxyguanosine
phenylacetylglycine
uridine 5'-monophosphate (UMP)
N6-carboxymethyllysine
uracil
1-palmitoyl-GPI (16:0)
chiro-inositol
glutamine
ceramide (d18:1/17:0, d17:1/18:0)*)
biochemicals_order = as.vector(ordered_all[,1])
biochemicals_order[1:5]

[1] "5-methyl-2'-deoxycytidine"   "glycocholate sulfate"       
[3] "myristoleate (14:1n5)"       "N-acetyl-3-methylhistidine*"
[5] "palmitoleoyl ethanolamide*"

In [20]:
pre = subset(df, AfterProbiotic == 0)[, 2:824]
pre = subset(pre, select=biochemicals_order)
print(ncol(pre))
pre_means = as.vector(colMeans(pre))
pre_means[1:5]

[1] 823


[1] 0.006088053 0.003140369 0.034112072 0.013613349 0.031938988

In [21]:
post = subset(df, AfterProbiotic == 1)[, 2:824]
post = subset(post, select=biochemicals_order)
print(ncol(post))
post_means = as.vector(colMeans(post))
post_means[1:5]

[1] 823


[1] 0.008384498 0.001202254 0.027995221 0.018672667 0.020593015

In [22]:
means = data.frame(
    means_pre = pre_means,
    means_post = post_means
)
output = cbind(ordered_all, means)
colnames(output) = c("Biochemical Name", "adj-P-value (BH)", "mean (Pre)", "mean (Post)")
head(output)

,Biochemical Name,adj-P-value (BH),mean (Pre),mean (Post)
344,5-methyl-2'-deoxycytidine,0.02028685,0.006088053,0.008384498
184,glycocholate sulfate,0.12446353,0.003140369,0.001202254
222,myristoleate (14:1n5),0.12446353,0.034112072,0.027995221
473,N-acetyl-3-methylhistidine*,0.12446353,0.013613349,0.018672667
696,palmitoleoyl ethanolamide*,0.12446353,0.031938988,0.020593015
710,pheophytin A,0.12446353,0.055106928,0.033595361


In [23]:
write.csv(output, '../output/p_vals_all_means.csv', row.names=FALSE)